In [4]:
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
from PIL import ImageDraw, ImageFont
from skimage.measure import label, regionprops
import cv2
import os

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return np.array(image)

def quantize_colors(image_array, n_colors):
    # Apply Gaussian blur to smooth transitions and reduce small color noise
    blurred = cv2.GaussianBlur(image_array, (5, 5), 0)

    h, w, c = blurred.shape
    image_flat = blurred.reshape(-1, 3)
    kmeans = KMeans(n_clusters=n_colors, random_state=42).fit(image_flat)
    labels = kmeans.labels_
    quantized_flat = kmeans.cluster_centers_[labels].astype('uint8')
    quantized_image = quantized_flat.reshape(h, w, 3)
    return quantized_image, labels.reshape(h, w), kmeans.cluster_centers_

def detect_regions_by_color(quantized_image):
    h, w, _ = quantized_image.shape
    labeled_image = np.zeros((h, w), dtype=int)
    current_label = 1

    unique_colors = np.unique(quantized_image.reshape(-1, 3), axis=0)
    for color in unique_colors:
        mask = np.all(quantized_image == color, axis=-1)
        labeled_mask = label(mask, connectivity=1)
        labeled_mask[labeled_mask > 0] += current_label - 1
        labeled_image += labeled_mask
        current_label = labeled_image.max() + 1

    return labeled_image

def filter_small_regions(labeled_image, min_size=100):
    new_labeled = np.zeros_like(labeled_image)
    current_label = 1
    for region in regionprops(labeled_image):
        if region.area >= min_size:
            coords = region.coords
            for y, x in coords:
                new_labeled[y, x] = current_label
            current_label += 1
    return new_labeled

def assign_numbers(cluster_centers):
    color_to_number = {tuple(center.astype(int)): idx+1 for idx, center in enumerate(cluster_centers)}
    return color_to_number

def save_legend(color_to_number, output_folder):
    legend_path = os.path.join(output_folder, "legend.txt")
    with open(legend_path, "w") as f:
        for color, number in color_to_number.items():
            f.write(f"{number}: RGB{color}\n")
    print(f"Legend saved to {legend_path}")

def overlay_numbers(image_array, labeled_regions, color_to_number):
    image = Image.fromarray(image_array)
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    for region_label in np.unique(labeled_regions):
        if region_label == 0:
            continue
        mask = labeled_regions == region_label
        coords = np.argwhere(mask)
        y, x = coords.mean(axis=0).astype(int)
        color = tuple(image_array[mask][0])
        number = color_to_number.get(color, '?')
        draw.text((x, y), str(number), fill='black', font=font)

    return image

def create_paint_by_numbers(image_path, save_path, n_colors):
    output_folder = save_path
    os.makedirs(output_folder, exist_ok=True)

    image_array = load_image(image_path)
    quantized_image, label_image, cluster_centers = quantize_colors(image_array, n_colors)
    labeled_regions = detect_regions_by_color(quantized_image)
    labeled_regions = filter_small_regions(labeled_regions, min_size=150)  # tweak size
    color_to_number = assign_numbers(cluster_centers)
    final_image = overlay_numbers(quantized_image, labeled_regions, color_to_number)

    # Save image
    output_image_path = os.path.join(output_folder, "paint_by_numbers_output.png")
    final_image.save(output_image_path)
    print(f"Paint-by-numbers image saved to {output_image_path}")

    # Save legend
    save_legend(color_to_number, output_folder)



In [5]:
image_path = '../test-images-mnz/castle.jpeg'
save_path = '../target'
number_colors = 16

create_paint_by_numbers(image_path, save_path, number_colors)

Paint-by-numbers image saved to ../target/paint_by_numbers_output.png
Legend saved to ../target/legend.txt
